In [1]:
%matplotlib inline

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.decomposition import TruncatedSVD, IncrementalPCA
from scipy.stats import chi2_contingency
from nltk.probability import FreqDist
from collections import defaultdict
from clpsych.store import Store
from clpsych.helpers import load_tokens

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse
import pandas as pd
import numpy as np
import codecs
import spacy
import glob

In [12]:
train_classes = pd.DataFrame.from_csv('data/classes/train_classes.txt')
dev_classes = pd.DataFrame.from_csv('data/classes/dev_classes.txt')
#sample_classes = pd.DataFrame.from_csv('data/classes/sample_classes.txt')

In [13]:
df = load_tokens(mask='data/tokens/lemmas.txt')

In [15]:
labeled_docs = df.merge(train_classes, on='post_id').replace(np.nan, '', regex=True)
test_docs = df.merge(dev_classes, on='post_id').replace(np.nan, '', regex=True)

In [16]:
labeled_docs['text_features'] = labeled_docs[['title', 'doc']].apply(lambda x: ' '.join(x), axis=1)
test_docs['text_features'] = test_docs[['title', 'doc']].apply(lambda x: ' '.join(x), axis=1)

In [17]:
# clear out the big dataframe -- now we only need the subsets
%reset_selective df

Once deleted, variables cannot be recovered. Proceed (y/[n])?  
Nothing done.


In [18]:
df['text_features'] = df[['title', 'doc']].astype(str).apply(lambda x: ' '.join(x), axis=1)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

vectorizer = CountVectorizer(
    tokenizer=lambda x: x.split(),
    stop_words='english', 
    decode_error='ignore', 
    min_df=40
)

transformer = TfidfTransformer()

In [25]:
vectorizer.fit(labeled_docs.text_features)
vocab = vectorizer.transform(labeled_docs.text_features)
transformer.fit(vocab)
vocab = transformer.transform(vocab)
test = transformer.transform(vectorizer.transform(test_docs.text_features))
total = transformer.transform(vectorizer.transform(df.text_features))
#test = vectorizer.transform(test_docs.title)

In [26]:
svd = TruncatedSVD(n_components=100, algorithm='arpack')
svd.fit(vocab)

TruncatedSVD(algorithm='arpack', n_components=100, n_iter=5,
       random_state=None, tol=0.0)

In [27]:
vocab = svd.transform(vocab)
test = svd.transform(test)
total = svd.transform(total)

In [28]:
%reset_selective vocab
%reset_selective test

pd.concat([df['post_id'], pd.DataFrame(total)], axis=1).to_csv('data/features/bow_100.csv')

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
